Navn: Ludvik Høibjerg-Aslaksen,
NMBU-mail: ludvik.hoibjerg-aslaksen@nmbu.no

Navn: Georg Dahl-Steinsvik,
NMBU-mail: georg.dahl-steinsvik@nmbu.no

In [ ]:
import meshio
from abc import ABC, abstractmethod

class Mesh:
    '''
    Represents a mesh structure, consisting of points and cells, and provides 
    functionality to identify neighboring cells within the mesh.
    '''
    def __init__(self) -> None:
        self._cell_id = -1 #id represents the index of a cell in _cells
        msh = meshio.read("simple.msh") #Reads the meshfile 
        #Generates a list containing point objects
        self._points = [Point(index, points[0], points[1], points[2]) for index, points in enumerate(msh.points)]
        #Generates a list containing cell objects of the type line or triangle
        self._cells = [self.cell_factory(cell) for cell in msh.cells[0].data] + [self.cell_factory(cell) for cell in msh.cells[1].data] #A list containing cell objects

    def cell_factory(self, cell: list[int]) -> object:
        self._cell_id = self._cell_id + 1
        cell_check = len(cell)
        cell_map = {
            2:Line,
            3:Triangle
        }

        return cell_map[cell_check](cell, self._points, self._cell_id)
    
    def find_neighbors(self, cell_id) -> None:
        neighboring_cells = []
        current_cell = self._cells[cell_id]
        current_cell_points = self._cells[cell_id].points()

        #Checks triangles for neighbors
        if len(current_cell_points) == 3:
            #The points of the current cell
            point_0 = current_cell_points[0].index()
            point_1 = current_cell_points[1].index()
            point_2 = current_cell_points[2].index()
            for cells in self._cells:
                point_indexes = [points.index() for points in cells.points()]
                #Checks if two different cells has exactly two points equal eachother. 
                #This is done by taking the sum of three boolean statements that are true if the cell, 
                #whose neighbors are being checked, have a point that is equal in the cell being checked.
                if sum((point_0 in point_indexes, point_1 in point_indexes, point_2 in point_indexes)) == 2:
                    neighboring_cells.append(cells)
        else:
            #The points of the current cell
            point_0 = current_cell_points[0].index()
            point_1 = current_cell_points[1].index()
            for cells in self._cells:
                point_indexes = [points.index() for points in cells.points()]
                #Checks if two different cells has exactly two points equal eachother. 
                #This is done by taking the sum of three boolean statements that are true if the cell, 
                #whose neighbors are being checked, have a point that is equal in the cell being checked.
                if sum((point_0 in point_indexes, point_1 in point_indexes)) == 1 and type(cells) is Line:
                    neighboring_cells.append(cells)

                elif sum((point_0 in point_indexes, point_1 in point_indexes)) == 2 and type(cells) is Triangle:
                    neighboring_cells.append(cells)
        current_cell.store_neighbors(neighboring_cells)
            
    #Outputs the list of points
    def points(self) -> list[object]:
        return self._points

    #Outputs the list of cells
    def cells(self) -> list[object]:
        return self._cells

class Cell(ABC):
    '''
    Abstract class representing a generic cell in a mesh
    '''
    @abstractmethod
    def __init__(self, cell: list[int], point_list: list[object], cell_id: int) -> None:
        self._neighbors = []
        self._id = cell_id
        #self._points_in_cell to be defined in each different type of cell. 
        #It should look something like this: self._points_in_cell = (point_list[cell[0]], point_list[cell[1]], point_list[cell[2]])
        self._points_in_cell = () 

    #Outputs a tuple with each point as an object
    def points(self) -> tuple[object]:
        return self._points_in_cell 

    #Outputs the id of a cell
    def id(self) -> int:
        return self._id 
    
    #Stores the neighbors found in in find_neighbors
    def store_neighbors(self, neighboring_cells):
        self._neighbors = neighboring_cells

    @abstractmethod
    def __str__(self) -> str:
        pass

class Triangle(Cell):
    '''
    A triangle cell in a mesh represented by three points. This class includes __str__ method for printing
    neighbors, the id of the current cell and if the cell is a boundry
    '''
    def __init__(self, cell: list[int], point_list: list[object], cell_id: int) -> None:
        super().__init__(cell, point_list, cell_id)
        self._points_in_cell = (point_list[cell[0]], point_list[cell[1]], point_list[cell[2]])
    
    def __str__(self) -> str:
        #Checks the if the neighbors are lines then returns true if they are
        return  f"Cell {self._id}:\nNeighbors: {[neighbors.id() for neighbors in self._neighbors]}\nThe cell {self._id} is boundry: {sorted([True if type(neighbors)==Line else False for neighbors in self._neighbors], reverse=True)[0]}\n"

class Line(Cell):
    '''
    A line cell in a mesh represented by two points. This class includes __str__ method for printing
    neighbors, the id of the current cell and the cell is always a boundry since it is a line
    '''
    def __init__(self, cell: list[int], point_list: list[object], cell_id: int) -> None:
        super().__init__(cell, point_list, cell_id)
        self._points_in_cell = (point_list[cell[0]], point_list[cell[1]])

    def __str__(self) -> str:
        #Boundry always returns True since it the cell is a line
        return  f"Cell {self._id}:\nNeighbors: {[neighbors.id() for neighbors in self._neighbors]}\nThe cell {self._id} is boundry: {True}\n"

class Point:
    '''
    A point class which contains points with x, y, z coordinates, 
    and the index of the point in list of points from the mesh class.
    '''
    def __init__(self, index: int, point_x: float, point_y: float, point_z: float) -> None:
        self._point_coordinates = (point_x, point_y, point_z)
        self._point_index = index

    #Gives the coord as a tuple 
    def coords(self) -> tuple[float]:
        return self._point_coordinates

    #Gives the index of the point object in mesh point list
    def index(self) -> int:
        return self._point_index

m = Mesh()
m.find_neighbors(4)
m.find_neighbors(189)
m.find_neighbors(222)
print(f"{m.cells()[4]}\n{m.cells()[189]}\n{m.cells()[222]}")



0
